In [1]:
%matplotlib

import re
import copy
import sys
from enum import Enum
import matplotlib.pyplot as plt
import time
import numpy as np
import random
import plotly.figure_factory as ff

Using matplotlib backend: Qt5Agg


In [146]:
instances_path = 'JSPLIB/instances/'
instance_path = instances_path + 'ft06'

def SPT(possible_tasks):
    smallest_duration = sys.maxsize
    best_task = None
    for t in possible_tasks:
        if t.duration < smallest_duration:
            smallest_duration = t.duration
            best_task = t
    return best_task

def LPT(possible_tasks):
    smallest_duration = 0
    best_task = None
    for t in possible_tasks:
        if t.duration > smallest_duration:
            smallest_duration = t.duration
            best_task = t
    return best_task

class STATE(Enum):
    NOT_DONE = '--'
    ON_GOING = '+/-' 
    DONE = '++'
   
class Task:   
    def __init__(self,ressource,duration,job_num):
        self.res = ressource
        self.job_num = job_num
        self.duration = duration
        self.state = STATE.NOT_DONE
        self.start_time = None
        self.possible_start_time = 0 #Utile pour la contruction du Gantt
        
    def reset(self):
        self.state = STATE.NOT_DONE
        self.start_time = None
        self.possible_start_time = 0
        
    def to_string(self):
        return str(self.res) + " " + str(self.duration)

    def start(self,time):
        self.start_time = time
        self.state = STATE.ON_GOING
        
    def start_possible_time(self):
        self.start_time = self.possible_start_time
                
    def update_possible_start_time(self,time):
        if(self.possible_start_time < time):
            self.possible_start_time = time
    
    def update(self,time):
        if(self.state == STATE.ON_GOING):
            if time >= self.start_time + self.duration:
                self.state = STATE.DONE
                return self.res
        return -1
                
    def is_done(self):
        return self.state == STATE.DONE
        
class Job:
    def __init__(self,job_num):
        self.tasks = []
        self.num = job_num
        
    def reset(self):
        for t in self.tasks:
            t.reset()
        
    def init_possible_start(self):
        delay = 0
        for t in self.tasks:
            t.possible_start= delay
            delay = delay + t.duration
        
    def add_task(self,task):
        self.tasks.append(task)
        
    def to_string(self):
        res = "|"
        for t in self.tasks:
            res = res + " " + '{:^6}'.format(t.to_string()) + " |" 
        return res
    
    def get_next_task(self):
        on_going = [task for task in self.tasks if task.state == STATE.ON_GOING]
        if  len(on_going) == 0:
            not_done_tasks = [task for task in self.tasks if task.state == STATE.NOT_DONE]
            if(len(not_done_tasks) != 0):
                return not_done_tasks[0]
            else:
                return None
        
    def start_next_task(self,time):
        t = next(task for task in self.tasks if task.state == STATE.NOT_DONE)
        if t != None:
            t.start(time)
            return t.res
        
    def start_task_at_possible_time(self,task_ind):
        t = self.tasks[task_ind]
        t.start_possible_time()
        delay=t.duration + t.start_time
        for i in range(task_ind+1, len(self.tasks)):
            self.tasks[i].update_possible_start_time(delay)
            delay = delay + self.tasks[i].duration
        return t.res, t.start_time, t.duration+t.start_time
    
    def update_possible_time_by_ressource(self,ressource,time):
        for t in self.tasks:
            if(t.res == ressource):
                t.update_possible_start_time(time)
        
    def update(self,time):
        res_to_free = []
        for t in self.tasks:
            res_to_free.append(t.update(time))
        return res_to_free
            
    def is_done(self):
        done = True
        for t in self.tasks:
            done = done & t.is_done()
        return done
    
    def get_end_time(self):
        return self.tasks[-1].start_time + self.tasks[-1].duration
        
class Instance_heuristic:
    def __init__(self,n_machines,jobs):
        self.n_machines = n_machines   
        self.jobs=jobs
        self.machines_availability = [True]*n_machines
        self.time = 0
        self.repetition_order = []

    def get_max_tasks(self):
        nb_max=0
        for j in self.jobs:
            if(len(j.tasks)>nb_max):
                nb_max = len(j.tasks)
        return nb_max
    
    def check_possible_tasks(self):
        possible_tasks = []
        for j in self.jobs:
            next_task = j.get_next_task()
            if(next_task != None):
                if(self.machines_availability[next_task.res]):
                    possible_tasks = possible_tasks + [next_task]
        return possible_tasks
    
    def display_jobs(self):
        max_tasks_per_job = 0
        for j in self.jobs:
            if(max_tasks_per_job < len(j.tasks)):
                max_tasks_per_job = len(j.tasks)
        header=" "
        for t in range(max_tasks_per_job):
            header = header + " " + '{:^6}'.format("R D") + "  " 
        print(header)
        for j in self.jobs:
            print(j.to_string())
            
    def get_state_matrix(self):
        m = np.zeros((len(self.jobs),self.get_max_tasks()))
        for j_num,j in enumerate(self.jobs):
            red_tasks=[]
            orange_tasks=[]
            green_tasks=[]
            for i,t in enumerate(j.tasks):
                if(t.state == STATE.ON_GOING):     
                    m[j_num][i] = 0             
                elif(t.state == STATE.DONE):
                    m[j_num][i] = 1           
                else:
                    m[j_num][i] = -1 
        return m
            
        
    def get_all_tasks(self):
        all_tasks = []
        for j in self.jobs:
            all_tasks = all_tasks + j.tasks
        return all_tasks
    
    def time_forward(self):
        self.time = self.time + 1
        for j in self.jobs:
            res_to_free = j.update(self.time)
            
            for r in res_to_free:
                if(r != -1):
                    self.machines_availability[r] = True
        
            
    def start_job(self,job_num):
        res_used = self.jobs[job_num].start_next_task(self.time)
        self.machines_availability[res_used] = False
        
    def is_done(self):
        done = True
        for j in self.jobs:
            done = done & j.is_done()
        return done
    
    def start_heuristic(self,fct=SPT,rand=0,display=False):
        if(display):
            matrix_state = self.get_state_matrix()
            fig, ax = plt.subplots()
            grid_x=[i+0.5 for i in range(matrix_state.shape[1])]
            ax.set_xticks(grid_x,minor=True)
            grid_y=[i+0.5 for i in range(matrix_state.shape[0])]
            ax.set_yticks(grid_y,minor=True)
            ax.grid(which='minor')
            ax.matshow(matrix_state, cmap='RdYlGn', vmin=-1, vmax=1)
            for i,jb in enumerate(self.jobs):
                for j,t in enumerate(jb.tasks):
                    text = ax.text(j, i, t.res,ha="center", va="center", color='black', fontsize=16)
            plt.xlabel('Tasks')
            plt.ylabel('Jobs')
            
        while not self.is_done():
            possible_tasks = self.check_possible_tasks()
            if(len(possible_tasks) > 0):
                if(random.random()<rand):
                    highest_prio_task = random.choice(possible_tasks)
                else:
                    highest_prio_task = fct(possible_tasks)
                self.start_job(highest_prio_task.job_num)
                self.repetition_order.append(highest_prio_task.job_num)
            else:
                self.time_forward()
                
                if(display):
                    matrix_state = self.get_state_matrix()
                    ax.matshow(matrix_state, cmap='RdYlGn', vmin=-1, vmax=1)
                    ax.set_title("TIME : "+str(self.time))
                    plt.pause(0.01)
  


In [147]:
list_jobs = []
n_machines = 0
n_jobs = 0
# Lecture de l'instance
with open(instance_path,'r') as f:
    num_line = 0
    for line in f:
        if(line[0] != '#'):
            
            re_line = re.sub(' +',';',line)
            re_line = re.sub('\\n','',re_line)
            split_line = re_line.split(';')
            if (num_line == 0):
                n_machines = int(split_line[1])
                n_jobs = int(split_line[0])
            else:
                list_jobs.append(split_line)
            num_line = num_line+1
list_jobs

[['2', '1', '0', '3', '1', '6', '3', '7', '5', '3', '4', '6'],
 ['1', '8', '2', '5', '4', '10', '5', '10', '0', '10', '3', '4'],
 ['2', '5', '3', '4', '5', '8', '0', '9', '1', '1', '4', '7'],
 ['1', '5', '0', '5', '2', '5', '3', '3', '4', '8', '5', '9'],
 ['2', '9', '1', '3', '4', '5', '5', '4', '0', '3', '3', '1'],
 ['1', '3', '3', '3', '5', '9', '0', '10', '4', '4', '2', '1']]

In [148]:
# Création des jobs à partir des listes précédentes
jobs = []
for i in range(n_machines):
    job = Job(i)
    for j in range(0,2*n_jobs,2):
        job.add_task(Task(int(list_jobs[i][j]),int(list_jobs[i][j+1]),i))
    jobs.append(job)
    
instance = Instance_heuristic(n_machines,jobs)

In [149]:


print("Nombre de machines : ",instance.n_machines)
print('Nombre de jobs : ',len(instance.jobs))
print('Taches : ')
instance.display_jobs()

Nombre de machines :  6
Nombre de jobs :  6
Taches : 
   R D      R D      R D      R D      R D      R D    
|  2 1   |  0 3   |  1 6   |  3 7   |  5 3   |  4 6   |
|  1 8   |  2 5   |  4 10  |  5 10  |  0 10  |  3 4   |
|  2 5   |  3 4   |  5 8   |  0 9   |  1 1   |  4 7   |
|  1 5   |  0 5   |  2 5   |  3 3   |  4 8   |  5 9   |
|  2 9   |  1 3   |  4 5   |  5 4   |  0 3   |  3 1   |
|  1 3   |  3 3   |  5 9   |  0 10  |  4 4   |  2 1   |


In [150]:
instance.start_heuristic(fct=LPT,rand=0,display=False)
print(instance.repetition_order)

[4, 1, 3, 1, 3, 4, 1, 2, 5, 3, 2, 2, 5, 4, 0, 0, 3, 0, 3, 1, 2, 0, 2, 1, 3, 2, 5, 1, 5, 4, 0, 0, 4, 5, 4, 5]


In [151]:
print(instance.time)

77


In [169]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

class Instance_descente():
    def __init__(self,repetition_order,jobs):
        self.repetition_order = repetition_order
        self.jobs = jobs

    def compute(self,display_gantt=True):
        if(display_gantt):
            colors = {}
#             list_colors = list(matplotlib.colors.cnames.keys())
            list_colors = ['dodgerblue','seagreen','sienna','turquoise','orchid','gold','black','white']
            fig = go.Figure(
                layout = {
                    'barmode': 'stack',
                    'xaxis': {'automargin': True},
                    'yaxis': {'automargin': True}}#, 'categoryorder': 'category ascending'}}
            )
        job_task_index = [0] * len(self.jobs)
        for j in jobs:
            j.reset()
            j.init_possible_start()
        for i in self.repetition_order:
            ressource_used,start_time, end_time = jobs[i].start_task_at_possible_time(job_task_index[i])
            if(display_gantt):
                show_legend = False
                if(str(ressource_used) not in colors.keys()):
                    color_picked = list_colors[0]
                    list_colors.remove(color_picked)
                    colors[str(ressource_used)] =  color_picked
                    show_legend = True

                fig.add_bar(x=[end_time-start_time],
                    y=['Job ' + str(i)],
                    base=start_time,
                    orientation='h',
                    showlegend=show_legend,
                    name=str(ressource_used),
                    marker={'color': colors[str(ressource_used)]}) 

            
            job_task_index[i] = job_task_index[i] + 1
            for j in self.jobs:
                j.update_possible_time_by_ressource(ressource_used,end_time)

        if(display_gantt): iplot(fig)
   
    def get_end_time(self):
        max_jobs = []
        for j in self.jobs:
            max_jobs.append(j.get_end_time())
        return max(max_jobs)
            
    def swap(self,ind1,ind2):
        c = self.repetition_order[ind1]
        self.repetition_order[ind1] = self.repetition_order[ind2]
        self.repetition_order[ind2] = c

            
            
        
instance_descente = Instance_descente(instance.repetition_order.copy(),instance.jobs.copy())  
print('Repetition order from heuristic : ', instance_descente.repetition_order)
instance_descente.compute(display_gantt=True)
print('Time : ',instance_descente.get_end_time())
instance_descente.swap(1,2)
print('Repetitionn order with swap ''stance_descente.repetition_order)
instance_descente.compute(display_gantt=True)
print('Time : ',instance_descente.get_end_time())

[4, 1, 3, 1, 3, 4, 1, 2, 5, 3, 2, 2, 5, 4, 0, 0, 3, 0, 3, 1, 2, 0, 2, 1, 3, 2, 5, 1, 5, 4, 0, 0, 4, 5, 4, 5]


Time :  77
[4, 3, 1, 1, 3, 4, 1, 2, 5, 3, 2, 2, 5, 4, 0, 0, 3, 0, 3, 1, 2, 0, 2, 1, 3, 2, 5, 1, 5, 4, 0, 0, 4, 5, 4, 5]


Time :  81


In [154]:
import matplotlib
for name,hexa in matplotlib.colors.cnames.items():
    print(name,hexa)
 
    

aliceblue #F0F8FF
antiquewhite #FAEBD7
aqua #00FFFF
aquamarine #7FFFD4
azure #F0FFFF
beige #F5F5DC
bisque #FFE4C4
black #000000
blanchedalmond #FFEBCD
blue #0000FF
blueviolet #8A2BE2
brown #A52A2A
burlywood #DEB887
cadetblue #5F9EA0
chartreuse #7FFF00
chocolate #D2691E
coral #FF7F50
cornflowerblue #6495ED
cornsilk #FFF8DC
crimson #DC143C
cyan #00FFFF
darkblue #00008B
darkcyan #008B8B
darkgoldenrod #B8860B
darkgray #A9A9A9
darkgreen #006400
darkgrey #A9A9A9
darkkhaki #BDB76B
darkmagenta #8B008B
darkolivegreen #556B2F
darkorange #FF8C00
darkorchid #9932CC
darkred #8B0000
darksalmon #E9967A
darkseagreen #8FBC8F
darkslateblue #483D8B
darkslategray #2F4F4F
darkslategrey #2F4F4F
darkturquoise #00CED1
darkviolet #9400D3
deeppink #FF1493
deepskyblue #00BFFF
dimgray #696969
dimgrey #696969
dodgerblue #1E90FF
firebrick #B22222
floralwhite #FFFAF0
forestgreen #228B22
fuchsia #FF00FF
gainsboro #DCDCDC
ghostwhite #F8F8FF
gold #FFD700
goldenrod #DAA520
gray #808080
green #008000
greenyellow #ADFF2F


In [27]:
a[2:]

[4, 5]

None
